In [ ]:
import pandas as pd

In [ ]:
!pip install sentence-transformers
!pip install neo4j

In [ ]:
from sentence_transformers import SentenceTransformer
from neo4j import GraphDatabase
from IPython.display import display
from IPython.display import Markdown

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/receitas_dataset_atualizado11.csv')
df

,Nome da Receita,País,Ingredientes,Modo de Preparo,Porção,Rendimento,Tempo de Preparo,Calorias,Proteínas,Carboidratos,Gorduras,Fibras,Restrições Alimentares
0,Ratatouille,França,"Abobrinha, Berinjela, Tomate, Pimentão, Cebola...",1. Corte os legumes em rodelas. 2. Refogue alh...,1 prato,4 porções,50 minutos,130,2,18,6,4,"Vegana, Sem Glúten, Halal"
1,Feijoada,Brasil,"Feijão preto, carne seca, linguiça, costelinha...",1. Dessalgue as carnes. 2. Cozinhe o feijão co...,1 prato,6 porções,2 horas,550,30,40,28,10,Sem Glúten
2,Yakisoba Vegan,Japão,"Macarrão de arroz, repolho, cenoura, brócolis,...",1. Cozinhe o macarrão. 2. Refogue legumes. 3. ...,1 prato,3 porções,40 minutos,300,8,45,10,6,"Vegana, Sem Lactose, Halal, Kosher"
3,Camarão com caldo de moqueca,Brasil,"Camarão, azeite, suco do limão, 1 dente de alh...",1. Temperar o camarão: Camarão + limão + alho ...,1 prato,3 porções,40 minutos,400,25,25,25,4,"Sem Lactose, camarão – alergênico"
4,Ravioli de Camarão com Massa Caseira,Itália,"2 xícaras de farinha de trigo, 2 ovos, 1 colhe...","\n 1. Fazer a massa: misturar farinha, ...",1 prato,3 porções,1h20 minutos,500,25,50,20,2,"Contém glúten, ovos e frutos do mar; pode cont..."
5,Brownie de Limão,Estados Unidos,"1 xícara de farinha de trigo, 200g de chocolat...",\n 1. Derreter o chocolate e a manteiga j...,1 fatia,8 porções,45 minutos,375,5,48,22,2,"Contém glúten, ovos e leite; pode conter traço..."
6,Boeuf Bourguignon,França,"1 kg de carne bovina para cozido, 150g de baco...",\n 1. Cortar a carne em cubos e dourar co...,1 prato,4 porções,3 horas,550,45,15,35,3,"Contém glúten (farinha), contém carne bovina e..."
7,Pavlova,Austrália / Nova Zelândia,"4 claras de ovo, 1 xícara de açúcar, 1 colher ...",\n 1. Preaquecer o forno a 120°C.\n ...,1 fatia,6 porções,1h40 minutos,350,6,60,10,2,Contém ovos e lactose
8,Berinjela Mediterrânea,Grécia,"2 berinjelas médias, 2 tomates maduros, 1 cebo...",\n 1. Cortar as berinjelas em fatias ou c...,1 prato,4 porções,40 minutos,180,5,15,12,7,Vegetariana; contém lactose se usar queijo feta
9,Nhoque à Bolonhesa,Itália,"500g de batata, 150g de farinha de trigo, 1 ov...","\n 1. Cozinhar as batatas, amassar e deix...",1 prato,4 porções,1h20 minutos,600,30,70,20,4,"Contém glúten, ovos e carne bovina"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Nome da Receita         11 non-null     object
 1   País                    11 non-null     object
 2   Ingredientes            11 non-null     object
 3   Modo de Preparo         11 non-null     object
 4   Porção                  11 non-null     object
 5   Rendimento              11 non-null     object
 6   Tempo de Preparo        11 non-null     object
 7   Calorias                11 non-null     int64 
 8   Proteínas               11 non-null     int64 
 9   Carboidratos            11 non-null     int64 
 10  Gorduras                11 non-null     int64 
 11  Fibras                  11 non-null     int64 
 12  Restrições Alimentares  11 non-null     object
dtypes: int64(5), object(8)
memory usage: 1.2+ KB


In [ ]:
from google.colab import userdata
uri_neo4j = userdata.get('Banco_neo4j')
key_neo4j = userdata.get('neo4j_key')

In [ ]:
uri = uri_neo4j
user = "neo4j"
password = key_neo4j

try:
    driver = GraphDatabase.driver(uri, auth=(user, password))
    driver.verify_connectivity()
    print("Connected successfully!")
except Exception as e:
    print(f"Connection error: {e}")

Connected successfully!


In [ ]:
def compile_text(x):

    text = f"""Receita: {x['Nome da Receita']},
            Ingredientes: {x['Ingredientes']},
            Modo de preparo: {x['Modo de Preparo']},
            Tempo de preparo = {x["Tempo de Preparo"]},
            Rendimento = {x["Rendimento"]},
            Calorias = {x["Calorias"]},
            Proteinas = {x["Proteínas"]},
            Carboidratos = {x["Carboidratos"]},
            Gorduras = {x["Gorduras"]},
            Fibras = {x["Fibras"]},
            Restricoes = {x["Restrições Alimentares"]}
            """

    return text

In [ ]:
sentences = df.apply(lambda x: compile_text(x), axis=1).tolist()
sentences

['Receita: Ratatouille,\n            Ingredientes: Abobrinha, Berinjela, Tomate, Pimentão, Cebola, Alho, Azeite, Sal, Ervas finas, \n            Modo de preparo: 1. Corte os legumes em rodelas. 2. Refogue alho e cebola no azeite. 3. Monte camadas em um refratário. 4. Asse por 40 min a 180ºC.,\n            Tempo de preparo = 50 minutos,\n            Rendimento = 4 porções,\n            Calorias = 130,\n            Proteinas = 2,\n            Carboidratos = 18,\n            Gorduras = 6,\n            Fibras = 4,  \n            Restricoes = Vegana, Sem Glúten, Halal\n            ',
 'Receita: Feijoada,\n            Ingredientes: Feijão preto, carne seca, linguiça, costelinha, cebola, alho, louro, sal, pimenta, \n            Modo de preparo: 1. Dessalgue as carnes. 2. Cozinhe o feijão com os temperos. 3. Adicione as carnes e cozinhe por 1h30.,\n            Tempo de preparo = 2 horas,\n            Rendimento = 6 porções,\n            Calorias = 550,\n            Proteinas = 30,\n           

In [ ]:
modelB = SentenceTransformer("all-MiniLM-L6-v2")
output = modelB.encode(sentences=sentences,
         show_progress_bar=True,
         normalize_embeddings=True)

embeddings = pd.DataFrame(output)
embeddings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.011004,-0.029464,-0.028500,-0.022270,-0.026840,0.019773,-0.006472,0.075718,0.008183,0.026409,...,-0.075986,-0.010424,0.050336,-0.059467,0.081915,-0.052442,0.004173,0.016064,0.045280,0.024100
1,-0.045122,-0.033156,-0.016308,0.051797,-0.051158,0.072475,0.051024,0.097697,-0.043209,-0.023275,...,-0.090344,-0.014226,-0.010673,-0.031168,0.059495,-0.095171,-0.022440,-0.002255,0.022751,0.030253
2,-0.051354,-0.026982,-0.041520,-0.011987,-0.028198,0.018968,0.018664,0.018743,-0.024292,-0.016694,...,-0.094686,0.003469,0.053130,-0.108267,0.056349,-0.053375,0.067057,-0.032392,0.048075,0.047834
3,-0.015343,-0.024228,-0.026911,-0.008341,0.033472,0.040192,0.016594,0.076914,-0.000075,0.067646,...,-0.044358,-0.036374,0.089830,-0.021538,0.025971,0.018705,0.010445,0.010424,-0.012551,-0.026869
4,-0.038626,-0.031303,-0.050835,-0.023732,-0.006038,0.019812,0.015446,0.121507,0.007143,-0.031990,...,-0.031949,-0.020763,0.055836,-0.010786,0.067128,-0.045233,0.049378,0.023082,-0.011340,-0.044295
5,-0.029761,-0.033854,-0.023954,0.024710,0.014215,0.011943,0.019873,0.061887,0.036223,0.002594,...,0.006505,-0.080730,0.024631,-0.035716,0.010120,-0.071659,0.086624,0.016991,-0.028409,-0.016821
6,-0.068739,0.001794,-0.047142,-0.014321,-0.095425,0.014476,-0.022066,0.048475,0.009975,-0.038781,...,-0.042287,-0.001228,0.022312,-0.008955,0.122318,-0.065921,-0.030471,0.058113,0.041803,-0.035941
7,-0.056717,0.001454,-0.022682,-0.012863,-0.072553,0.064163,-0.026445,0.069605,-0.005040,0.018712,...,-0.074932,-0.033285,0.025258,-0.006865,0.065337,-0.050489,0.059790,-0.035122,-0.006795,-0.023486
8,-0.017057,0.023477,0.011307,-0.006366,-0.050898,0.003533,-0.043083,0.061086,0.029535,0.039007,...,-0.041198,-0.050682,0.043155,0.040158,0.065047,-0.036361,0.028402,0.062916,0.015800,-0.037471
9,-0.030933,0.036316,-0.059272,-0.051452,-0.031512,-0.008104,-0.011270,0.105761,0.017766,0.075416,...,-0.085008,-0.014766,-0.017175,-0.035255,0.096453,-0.072309,0.068916,-0.000081,0.008619,-0.041353


In [ ]:
df["Embedding"] = embeddings.values.tolist()
df

,Nome da Receita,País,Ingredientes,Modo de Preparo,Porção,Rendimento,Tempo de Preparo,Calorias,Proteínas,Carboidratos,Gorduras,Fibras,Restrições Alimentares,Embedding
0,Ratatouille,França,"Abobrinha, Berinjela, Tomate, Pimentão, Cebola...",1. Corte os legumes em rodelas. 2. Refogue alh...,1 prato,4 porções,50 minutos,130,2,18,6,4,"Vegana, Sem Glúten, Halal","[-0.011004406027495861, -0.02946416847407818, ..."
1,Feijoada,Brasil,"Feijão preto, carne seca, linguiça, costelinha...",1. Dessalgue as carnes. 2. Cozinhe o feijão co...,1 prato,6 porções,2 horas,550,30,40,28,10,Sem Glúten,"[-0.04512203112244606, -0.033156394958496094, ..."
2,Yakisoba Vegan,Japão,"Macarrão de arroz, repolho, cenoura, brócolis,...",1. Cozinhe o macarrão. 2. Refogue legumes. 3. ...,1 prato,3 porções,40 minutos,300,8,45,10,6,"Vegana, Sem Lactose, Halal, Kosher","[-0.05135401338338852, -0.026981506496667862, ..."
3,Camarão com caldo de moqueca,Brasil,"Camarão, azeite, suco do limão, 1 dente de alh...",1. Temperar o camarão: Camarão + limão + alho ...,1 prato,3 porções,40 minutos,400,25,25,25,4,"Sem Lactose, camarão – alergênico","[-0.015342619270086288, -0.024227507412433624,..."
4,Ravioli de Camarão com Massa Caseira,Itália,"2 xícaras de farinha de trigo, 2 ovos, 1 colhe...","\n 1. Fazer a massa: misturar farinha, ...",1 prato,3 porções,1h20 minutos,500,25,50,20,2,"Contém glúten, ovos e frutos do mar; pode cont...","[-0.03862620145082474, -0.031302694231271744, ..."
5,Brownie de Limão,Estados Unidos,"1 xícara de farinha de trigo, 200g de chocolat...",\n 1. Derreter o chocolate e a manteiga j...,1 fatia,8 porções,45 minutos,375,5,48,22,2,"Contém glúten, ovos e leite; pode conter traço...","[-0.02976149506866932, -0.03385382145643234, -..."
6,Boeuf Bourguignon,França,"1 kg de carne bovina para cozido, 150g de baco...",\n 1. Cortar a carne em cubos e dourar co...,1 prato,4 porções,3 horas,550,45,15,35,3,"Contém glúten (farinha), contém carne bovina e...","[-0.06873932480812073, 0.0017939272802323103, ..."
7,Pavlova,Austrália / Nova Zelândia,"4 claras de ovo, 1 xícara de açúcar, 1 colher ...",\n 1. Preaquecer o forno a 120°C.\n ...,1 fatia,6 porções,1h40 minutos,350,6,60,10,2,Contém ovos e lactose,"[-0.05671690031886101, 0.0014543916331604123, ..."
8,Berinjela Mediterrânea,Grécia,"2 berinjelas médias, 2 tomates maduros, 1 cebo...",\n 1. Cortar as berinjelas em fatias ou c...,1 prato,4 porções,40 minutos,180,5,15,12,7,Vegetariana; contém lactose se usar queijo feta,"[-0.01705687679350376, 0.023476798087358475, 0..."
9,Nhoque à Bolonhesa,Itália,"500g de batata, 150g de farinha de trigo, 1 ov...","\n 1. Cozinhar as batatas, amassar e deix...",1 prato,4 porções,1h20 minutos,600,30,70,20,4,"Contém glúten, ovos e carne bovina","[-0.03093346580862999, 0.03631582111120224, -0..."


In [ ]:
def create_nodes_and_relationships(df):
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")

        for _, row in df.iterrows():
            session.run(
                """
                MERGE (r:Receita {
                    nome: $nome,
                    tempo_preparo: $tempo_preparo,
                    porcao: $porcao,
                    rendimento: $rendimento,
                    ingredientes: $ingredientes,
                    modo_preparo: $modo_preparo,
                    calorias: $calorias,
                    proteinas: $proteinas,
                    carboidratos: $carboidratos,
                    gorduras: $gorduras,
                    fibras: $fibras,
                    restricoes: $restricoes
                })
                MERGE (em:Embedding {embedding: $embedding})
                MERGE (em)-[:VETOR_DENSO_DE]->(r)

                """,
                nome=row["Nome da Receita"],
                tempo_preparo=row["Tempo de Preparo"],
                porcao=row["Porção"],
                rendimento=row["Rendimento"],
                ingredientes=row["Ingredientes"],
                modo_preparo=row["Modo de Preparo"],
                calorias=int(row["Calorias"]),
                proteinas=int(row["Proteínas"]),
                carboidratos=int(row["Carboidratos"]),
                gorduras=int(row["Gorduras"]),
                fibras=int(row["Fibras"]),
                restricoes=row["Restrições Alimentares"],
                embedding=row["Embedding"]
            )
        print('Nodos e relacionamentos criados com sucesso!')


In [ ]:
df.fillna('')

,Nome da Receita,País,Ingredientes,Modo de Preparo,Porção,Rendimento,Tempo de Preparo,Calorias,Proteínas,Carboidratos,Gorduras,Fibras,Restrições Alimentares,Embedding
0,Ratatouille,França,"Abobrinha, Berinjela, Tomate, Pimentão, Cebola...",1. Corte os legumes em rodelas. 2. Refogue alh...,1 prato,4 porções,50 minutos,130,2,18,6,4,"Vegana, Sem Glúten, Halal","[-0.011004406027495861, -0.02946416847407818, ..."
1,Feijoada,Brasil,"Feijão preto, carne seca, linguiça, costelinha...",1. Dessalgue as carnes. 2. Cozinhe o feijão co...,1 prato,6 porções,2 horas,550,30,40,28,10,Sem Glúten,"[-0.04512203112244606, -0.033156394958496094, ..."
2,Yakisoba Vegan,Japão,"Macarrão de arroz, repolho, cenoura, brócolis,...",1. Cozinhe o macarrão. 2. Refogue legumes. 3. ...,1 prato,3 porções,40 minutos,300,8,45,10,6,"Vegana, Sem Lactose, Halal, Kosher","[-0.05135401338338852, -0.026981506496667862, ..."
3,Camarão com caldo de moqueca,Brasil,"Camarão, azeite, suco do limão, 1 dente de alh...",1. Temperar o camarão: Camarão + limão + alho ...,1 prato,3 porções,40 minutos,400,25,25,25,4,"Sem Lactose, camarão – alergênico","[-0.015342619270086288, -0.024227507412433624,..."
4,Ravioli de Camarão com Massa Caseira,Itália,"2 xícaras de farinha de trigo, 2 ovos, 1 colhe...","\n 1. Fazer a massa: misturar farinha, ...",1 prato,3 porções,1h20 minutos,500,25,50,20,2,"Contém glúten, ovos e frutos do mar; pode cont...","[-0.03862620145082474, -0.031302694231271744, ..."
5,Brownie de Limão,Estados Unidos,"1 xícara de farinha de trigo, 200g de chocolat...",\n 1. Derreter o chocolate e a manteiga j...,1 fatia,8 porções,45 minutos,375,5,48,22,2,"Contém glúten, ovos e leite; pode conter traço...","[-0.02976149506866932, -0.03385382145643234, -..."
6,Boeuf Bourguignon,França,"1 kg de carne bovina para cozido, 150g de baco...",\n 1. Cortar a carne em cubos e dourar co...,1 prato,4 porções,3 horas,550,45,15,35,3,"Contém glúten (farinha), contém carne bovina e...","[-0.06873932480812073, 0.0017939272802323103, ..."
7,Pavlova,Austrália / Nova Zelândia,"4 claras de ovo, 1 xícara de açúcar, 1 colher ...",\n 1. Preaquecer o forno a 120°C.\n ...,1 fatia,6 porções,1h40 minutos,350,6,60,10,2,Contém ovos e lactose,"[-0.05671690031886101, 0.0014543916331604123, ..."
8,Berinjela Mediterrânea,Grécia,"2 berinjelas médias, 2 tomates maduros, 1 cebo...",\n 1. Cortar as berinjelas em fatias ou c...,1 prato,4 porções,40 minutos,180,5,15,12,7,Vegetariana; contém lactose se usar queijo feta,"[-0.01705687679350376, 0.023476798087358475, 0..."
9,Nhoque à Bolonhesa,Itália,"500g de batata, 150g de farinha de trigo, 1 ov...","\n 1. Cozinhar as batatas, amassar e deix...",1 prato,4 porções,1h20 minutos,600,30,70,20,4,"Contém glúten, ovos e carne bovina","[-0.03093346580862999, 0.03631582111120224, -0..."


In [ ]:
def clean_nan_values(df):
    return df.fillna("")
df = clean_nan_values(df)

In [ ]:
create_nodes_and_relationships(df)

Nodos e relacionamentos criados com sucesso!


In [ ]:
def get_embedding(query):
    return modelB.encode(query).tolist()

In [ ]:
def search_similar_products(query):
    query_embedding = get_embedding(query)
    with driver.session() as session:
        result = session.run(
            """
            WITH $query_embedding AS queryVec
            MATCH (em:Embedding)-[:VETOR_DENSO_DE]->(r:Receita)
            WITH r, vector.similarity.cosine(queryVec, em.embedding) AS similarity
            WHERE similarity >= 0.4
            RETURN r.nome AS receita, similarity
            ORDER BY similarity DESC
            LIMIT 5
            """,
            query_embedding=query_embedding
        )
        return result.data()

In [ ]:
sentence = "Eu tenho berinjela, abobrinha e tomate, o que posso fazer com esses ingredientes?"
similar_products = search_similar_products(sentence)
similar_products

[{'receita': 'Berinjela Mediterrânea', 'similarity': 0.8042004704475403},
 {'receita': 'Ratatouille', 'similarity': 0.7927350401878357},
 {'receita': 'Feijoada', 'similarity': 0.7792800068855286},
 {'receita': 'Boeuf Bourguignon', 'similarity': 0.7778276205062866},
 {'receita': 'Yakisoba Vegan', 'similarity': 0.771772027015686}]

In [ ]:
sentence = "Estou fazendo uma dieta e gostaria de uma receita com menos de 400 calorias e mais de 15g de proteína."
similar_products = search_similar_products(sentence)
similar_products

[{'receita': 'Yakisoba Vegan', 'similarity': 0.7629751563072205},
 {'receita': 'Feijoada', 'similarity': 0.7427618503570557},
 {'receita': 'Ratatouille', 'similarity': 0.7419766187667847},
 {'receita': 'Bacalhau com Natas', 'similarity': 0.7382675409317017},
 {'receita': 'Camarão com caldo de moqueca', 'similarity': 0.7315303087234497}]

In [ ]:
sentence = "Quais receitas têm menos de 400 calorias e mais de 20g de proteína por porção?"
similar_products = search_similar_products(sentence)
similar_products

[{'receita': 'Bacalhau com Natas', 'similarity': 0.7289949655532837},
 {'receita': 'Feijoada', 'similarity': 0.7196017503738403},
 {'receita': 'Yakisoba Vegan', 'similarity': 0.7192314863204956},
 {'receita': 'Nhoque à Bolonhesa', 'similarity': 0.7186548709869385},
 {'receita': 'Ratatouille', 'similarity': 0.7136179208755493}]

In [ ]:
sentence = "Possuo em casa: feijão, arroz, carne em casa, qual receita eu poderia fazer?"
similar_products = search_similar_products(sentence)
similar_products

[{'receita': 'Nhoque à Bolonhesa', 'similarity': 0.7122252583503723},
 {'receita': 'Feijoada', 'similarity': 0.7100175619125366},
 {'receita': 'Berinjela Mediterrânea', 'similarity': 0.7042367458343506},
 {'receita': 'Camarão com caldo de moqueca', 'similarity': 0.7037292718887329},
 {'receita': 'Pavlova', 'similarity': 0.6961371898651123}]

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai

In [ ]:
Gemini_Key=userdata.get('gemini_key')

genai.configure(api_key=Gemini_Key)

In [ ]:
modelGemini = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
prompt = """Eu tenho berinjela, abobrinha e tomate, o que posso fazer com esses ingredientes?
          Resultado:
          {'receita': 'Berinjela Mediterrânea', 'similarity': 0.7847647666931152},
          {'receita': 'Ratatouille', 'similarity': 0.7762736082077026},
          {'receita': 'Boeuf Bourguignon', 'similarity': 0.7636440396308899},
          {'receita': 'Bacalhau com Natas', 'similarity': 0.7431693077087402},
          {'receita': 'Camarão com caldo de moqueca', 'similarity': 0.7340882420539856}

Com base nas avaliações dos produtos apresentados, analise cada um deles em relação à sua busca.
"""

In [ ]:
from IPython.display import display, Markdown

In [ ]:
response = modelGemini.generate_content(f"{prompt} ")
display(Markdown(response.text))

A busca por receitas usando berinjela, abobrinha e tomate retorna resultados com diferentes níveis de pertinência.  A análise individual de cada sugestão, considerando a semelhança (similarity score) é:

* **Berinjela Mediterrânea (Similarity: 0.78):**  Este é o resultado mais promissor.  Berinjela e tomate são ingredientes quase obrigatórios em muitas receitas mediterrâneas, e a abobrinha pode facilmente ser incorporada. A alta similaridade sugere que a receita provavelmente incluirá esses três ingredientes de forma significativa.

* **Ratatouille (Similarity: 0.77):**  O Ratatouille é um clássico que usa berinjela, abobrinha e tomate como ingredientes principais. A similaridade alta confirma que é uma ótima opção, provavelmente muito próxima da Berinjela Mediterrânea em termos de uso dos ingredientes.

* **Boeuf Bourguignon (Similarity: 0.76):**  Este é surpreendente. Boeuf Bourguignon é um guisado de carne francês. Embora possa incluir tomates, a berinjela e a abobrinha não são ingredientes tradicionais. A similaridade relativamente alta pode indicar uma variação da receita ou uma adaptação, mas é a opção menos óbvia entre as listadas. Pode ser uma sugestão baseada na presença do tomate, que é um ingrediente comum em diversos tipos de guisados.

* **Bacalhau com Natas (Similarity: 0.74):**  Este é um prato português que tradicionalmente não leva berinjela e abobrinha. A inclusão na lista sugere que o algoritmo encontrou alguma associação fraca, talvez relacionada ao uso de tomate em algum contexto similar de cozimento cremoso.  É uma sugestão improvável.

* **Camarão com caldo de moqueca (Similarity: 0.73):** Assim como o Bacalhau com Natas, este é um prato que não utiliza normalmente berinjela e abobrinha.  A presença de tomate no caldo da moqueca pode explicar a sua inclusão, mas é a menos provável das sugestões.


**Conclusão:**

As melhores opções são, sem dúvida, **Berinjela Mediterrânea** e **Ratatouille**.  Estas receitas provavelmente farão bom uso dos três ingredientes disponíveis.  As outras opções são menos prováveis de serem adequadas, possivelmente devido a alguma associação fraca baseada no tomate, mas não na combinação dos três ingredientes.  Recomenda-se priorizar as duas primeiras sugestões.


In [ ]:
prompt = """
    Você é um assistente especializado em receitas culinárias, que conhece detalhes como ingredientes, modo de preparo, valores nutricionais, tempo e rendimento.
    Use as informações para responder perguntas sobre receitas, ingredientes, restrições alimentares, tempo de preparo e sugestões.
    Responda de forma clara e direta. Caso a pergunta envolva ingredientes específicos, sugira receitas que usem esses ingredientes.
    Sempre informe o nome da receita e detalhes importantes (tempo, porção, principais nutrientes).
"""

In [ ]:
pergunta = "Tem alguma receita com poucas calorias e que tenha alto teor de proteínas?"

In [ ]:
prompt = f"""{prompt} Pergunta: {pergunta} de acordo com {df}"""

response = modelGemini.generate_content(prompt)
display(Markdown(response.text))

Das receitas listadas, a que melhor se encaixa na sua busca por uma opção com poucas calorias e alto teor de proteínas é o **Boeuf Bourguignon**.

**Nome da Receita:** Boeuf Bourguignon
**País:** França
**Tempo de Preparo:** 3 horas
**Porção:** 1 prato
**Rendimento:** 4 porções
**Calorias:** 550 kcal (por porção)
**Proteínas:** 45g (por porção)
**Carboidratos:** 15g (por porção)
**Gorduras:** 35g (por porção)
**Fibras:** 3g (por porção)
**Restrições Alimentares:** Contém glúten (farinha), contém carne bovina e possivelmente outros alergênicos dependendo dos ingredientes adicionais.


**Observação importante:** Embora o Boeuf Bourguignon tenha alto teor de proteína, o conteúdo de gordura também é alto.  Se você busca uma opção ainda mais leve, considere diminuir a quantidade de gordura usada no preparo, substituindo, por exemplo, parte do bacon por cogumelos ou utilizando um corte de carne magra.  A receita de Ratatouille também é uma boa opção com poucas calorias, mas seu teor de proteína é significativamente menor.


In [ ]:
prompt = """Eu tenho berinjela, abobrinha e tomate, o que posso fazer com esses ingredientes?
          Resultado:
          {'receita': 'Berinjela Mediterrânea', 'similarity': 0.8042004704475403},
          {'receita': 'Ratatouille', 'similarity': 0.7927350401878357},
          {'receita': 'Feijoada', 'similarity': 0.7792800068855286},
          {'receita': 'Boeuf Bourguignon', 'similarity': 0.7778276205062866},
          {'receita': 'Yakisoba Vegan', 'similarity': 0.771772027015686}

Com base nas avaliações dos produtos apresentados, analise cada um deles em relação à sua busca.
"""

In [ ]:
response = modelGemini.generate_content(f"{prompt}, para a receita mais semelhante, gere a receita passo a passo de acordo com {df}")
display(Markdown(response.text))

A análise das receitas considerando a busca por receitas com berinjela, abobrinha e tomate, baseada nas similaridades e ingredientes, indica o seguinte:


**1. Ratatouille (Similaridade: 0.79):**  Esta é a melhor opção, de acordo com os dados fornecidos. A similaridade alta se justifica pela presença explícita de abobrinha, berinjela e tomate nos ingredientes.  O modo de preparo é simples e, aparentemente, delicioso.

**Receita passo a passo do Ratatouille:**

**Ingredientes:**

* 1 berinjela média, cortada em cubos
* 1 abobrinha média, cortada em cubos
* 2 tomates grandes, picados
* 1 cebola média, picada
* 1 pimentão vermelho médio, picado (pode ser opcional se não tiver)
* 2 dentes de alho, picados
* 2 colheres de sopa de azeite de oliva
* Orégano, manjericão e tomilho a gosto
* Sal e pimenta-do-reino a gosto

**Modo de Preparo:**

1. **Preparação dos legumes:** Lave bem todos os legumes e corte-os conforme descrito acima.  Se desejar, você pode retirar as sementes da berinjela para reduzir a umidade.
2. **Refogar:** Em uma panela grande ou frigideira funda, aqueça o azeite de oliva em fogo médio. Adicione a cebola e o alho e refogue até ficarem macios e translúcidos (aproximadamente 5 minutos).
3. **Adicionar os legumes:** Adicione a berinjela e a abobrinha à panela e cozinhe por cerca de 10 minutos, mexendo ocasionalmente, até que comecem a amolecer.
4. **Incorporar o tomate e temperos:** Adicione os tomates picados, o pimentão (se usar), o orégano, o manjericão, o tomilho, sal e pimenta-do-reino. Misture bem.
5. **Cozinhar:** Reduza o fogo para baixo, tampe a panela e cozinhe por 20 a 30 minutos, ou até que os legumes estejam macios e o molho tenha engrossado. Mexa de vez em quando para evitar que grude.
6. **Finalização:**  Prove e ajuste os temperos conforme o seu paladar. Sirva quente como acompanhamento ou prato principal.


**2. Berinjela Mediterrânea (Similaridade: 0.80):**  A similaridade é ainda maior do que a do Ratatouille, e isso sugere uma receita muito próxima ao que você busca,  embora os ingredientes específicos não estejam listados.  A receita provavelmente usará os três ingredientes principais de maneira similar ao Ratatouille, porém com um toque mediterrâneo nos temperos.

**3. As demais receitas (Feijoada, Boeuf Bourguignon, Yakisoba Vegan):** As similaridades são significativamente menores.  Embora possam incluir algum dos ingredientes da sua lista, elas não são focadas em berinjela, abobrinha e tomate como as duas primeiras opções.


**Conclusão:**  Para aproveitar seus ingredientes de forma mais eficiente e saborosa, o Ratatouille é a escolha mais recomendável, seguindo de perto a Berinjela Mediterrânea, caso você deseje algo com um perfil de sabor mais específico.


In [ ]:
prompt2 = """Estou fazendo uma dieta e gostaria de uma receita com menos de 400 calorias e mais de 15g de proteína.
          Resultado:
          {'receita': 'Bacalhau com Natas', 'similarity': 0.7289949655532837},
          {'receita': 'Feijoada', 'similarity': 0.7196017503738403},
          {'receita': 'Yakisoba Vegan', 'similarity': 0.7192314863204956},
          {'receita': 'Nhoque à Bolonhesa', 'similarity': 0.7186548709869385},
          {'receita': 'Ratatouille', 'similarity': 0.7136179208755493}

Com base nas avaliações dos produtos apresentados, analise cada um deles em relação à sua busca.
"""

In [ ]:
response = modelGemini.generate_content(f"{prompt2}, para a receita mais semelhante, gere a receita passo a passo de acordo com {df}")
display(Markdown(response.text))

A sua busca é por uma receita com menos de 400 calorias e mais de 15g de proteína. Vamos analisar as sugestões:

* **Bacalhau com Natas (450 calorias, 35g de proteína):**  Ultrapassa ligeiramente o limite de calorias (400), mas atende amplamente à necessidade de proteína.  Considerando a similaridade alta, seria uma boa opção se você pudesse ajustar a receita para reduzir as calorias, talvez diminuindo a quantidade de natas.

* **Feijoada (550 calorias, 30g de proteína):**  Excede significativamente o limite de calorias.  Não é uma boa opção para sua dieta.

* **Yakisoba Vegan (300 calorias, 8g de proteína):**  Atende ao limite de calorias, mas fica aquém da meta de proteína.

* **Nhoque à Bolonhesa (600 calorias, 30g de proteína):**  Excede significativamente o limite de calorias.  Não é uma opção adequada.

* **Ratatouille (130 calorias, 2g de proteína):**  Atende ao limite de calorias, mas tem proteína muito baixa.


**Conclusão:**  Das opções apresentadas, o **Bacalhau com Natas** é o mais próximo de atender aos seus critérios, apesar de ultrapassar um pouco o limite calórico.  Seria preciso adaptar a receita para reduzir as calorias.  As outras opções não atendem adequadamente a pelo menos um dos seus critérios.


**Receita Adaptada de Bacalhau com Natas (aproximada, para reduzir calorias):**  A receita original não foi fornecida, apenas os dados nutricionais.  Esta receita é uma adaptação com foco na redução calórica mantendo a proteína:

**Nome:** Bacalhau com Natas Light

**Porções:** 2
**Tempo de Preparo:** 45 minutos

**Ingredientes:**

* 200g de bacalhau dessalgado e desfiado (aproximadamente 20g de proteína)
* 100g de batata cozida e cortada em rodelas finas (para reduzir calorias e engrossar levemente o molho)
* 1/2 xícara de natas light (reduz significativamente calorias em comparação com natas tradicionais)
* 1 dente de alho picado
* 1/4 cebola picada
* 1 colher de sopa de azeite extra virgem
* Salsa picada a gosto
* Sal e pimenta do reino a gosto

**Modo de Preparo:**

1. Em uma panela, refogue o alho e a cebola no azeite até dourar levemente.
2. Adicione o bacalhau desfiado e misture bem.
3. Adicione as rodelas de batata cozida.
4. Junte as natas light e misture delicadamente.
5. Tempere com sal e pimenta do reino a gosto.
6. Cozinhe em fogo baixo por cerca de 10 minutos, mexendo ocasionalmente, até o molho engrossar levemente.
7. Sirva polvilhado com salsa picada.


**Observações:**

* Esta receita é uma estimativa e o valor calórico e de proteínas pode variar dependendo dos ingredientes usados e das quantidades.
* Para reduzir ainda mais as calorias, considere reduzir a quantidade de natas light e usar mais batata ou outros vegetais como espinafre ou brócolis.
* Use um aplicativo de contagem de calorias para calcular o valor nutricional da sua receita final.


Lembre-se de consultar um nutricionista para um plano alimentar personalizado e seguro para sua dieta.


In [ ]:
prompt3 = """Quais receitas têm menos de 400 calorias e mais de 20g de proteína por porção?
          Resultado:
          {'receita': 'Bacalhau com Natas', 'similarity': 0.7289949655532837},
          {'receita': 'Feijoada', 'similarity': 0.7196017503738403},
          {'receita': 'Yakisoba Vegan', 'similarity': 0.7192314863204956},
          {'receita': 'Nhoque à Bolonhesa', 'similarity': 0.7186548709869385},
          {'receita': 'Ratatouille', 'similarity': 0.7136179208755493}

Com base nas avaliações dos produtos apresentados, analise cada um deles em relação à sua busca.
"""

In [ ]:
response = modelGemini.generate_content(f"{prompt3}, para a receita mais semelhante, gere a receita passo a passo de acordo com {df}")
display(Markdown(response.text))

A lista apresentada contém sugestões de receitas com base em similaridade, porém **nenhuma** das receitas listadas atende simultaneamente aos critérios de menos de 400 calorias e mais de 20g de proteína por porção.

Vamos analisar cada uma:

* **Bacalhau com Natas:** Possui 450 calorias e 35g de proteína.  Ultrapassa o limite de calorias.
* **Feijoada:** Possui 550 calorias e 30g de proteína. Ultrapassa o limite de calorias.
* **Yakisoba Vegan:** Possui 300 calorias e 8g de proteína. Não atinge o mínimo de proteína.
* **Nhoque à Bolonhesa:** Possui 600 calorias e 30g de proteína. Ultrapassa o limite de calorias.
* **Ratatouille:** Possui 130 calorias e 2g de proteína. Não atinge o mínimo de proteína.


Para encontrar receitas que atendam aos seus critérios, será necessário realizar uma nova busca em um banco de dados de receitas, filtrando especificamente por:

* **Calorias:** Menor que 400
* **Proteína:** Maior que 20g

Lembre-se que as informações nutricionais podem variar dependendo dos ingredientes e métodos de preparo utilizados.  É importante consultar uma fonte confiável de informações nutricionais para ter certeza da composição de cada receita.


**Exemplo de como poderia ser uma receita que se encaixe nos seus critérios (precisaria ser verificada nutricionalmente):**

**Receita: Salmão com Espinafre e Quinoa** (Esta receita é uma sugestão e precisa ser calculada para verificar se atende aos critérios)

**Ingredientes:**

* 150g de salmão (filé)
* 1 xícara de quinoa cozida
* 1 xícara de espinafre fresco
* 1 colher de sopa de azeite
* Sal e pimenta a gosto

**Modo de Preparo:**

1. Tempere o salmão com sal e pimenta.
2. Aqueça o azeite em uma frigideira e cozinhe o salmão por cerca de 3-4 minutos de cada lado, até ficar cozido.
3. Adicione o espinafre à frigideira e cozinhe até murchar.
4. Sirva o salmão com a quinoa e o espinafre.

**Observação:**  Esta é apenas uma ideia. A quantidade precisa de cada ingrediente pode ser ajustada para controlar as calorias e a quantidade de proteína.  Você precisaria calcular o valor nutricional desta receita usando um rastreador de calorias online para confirmar se ela se encaixa nos seus critérios.  Outras fontes de proteína magra como frango grelhado ou peito de peru poderiam ser substituídas ao salmão.


In [ ]:
prompt4 = """Possuo em casa: feijão, arroz, carne em casa, qual receita eu poderia fazer?
          Resultado:
          {'receita': 'Nhoque à Bolonhesa', 'similarity': 0.7122252583503723},
          {'receita': 'Feijoada', 'similarity': 0.7100175619125366},
          {'receita': 'Berinjela Mediterrânea', 'similarity': 0.7042367458343506},
          {'receita': 'Camarão com caldo de moqueca', 'similarity': 0.7037292718887329},
          {'receita': 'Pavlova', 'similarity': 0.6961371898651123}

Com base nas avaliações dos produtos apresentados, analise cada um deles em relação à sua busca.
"""

In [ ]:
response = modelGemini.generate_content(f"{prompt4}, para a receita mais semelhante, gere a receita passo a passo de acordo com {df}")
display(Markdown(response.text))

A sua busca por receitas utilizando feijão, arroz e carne gerou as seguintes sugestões, ordenadas pela similaridade:

1. **Nhoque à Bolonhesa (similaridade: 0.712):**  Apesar de a receita não conter feijão explicitamente, a similaridade se deve provavelmente à presença da carne (na base de bolonhesa) e à possibilidade de adicionar feijão à receita como um ingrediente extra.  O arroz não é um ingrediente tradicional.

2. **Feijoada (similaridade: 0.710):** Esta é a receita mais óbvia e adequada, usando diretamente o feijão como ingrediente principal.  A carne também é fundamental na feijoada. O arroz, embora não essencial, é tradicionalmente servido como acompanhamento.

3. **Berinjela Mediterrânea (similaridade: 0.704):**  A carne não está presente nessa receita, e a similaridade pode ser atribuída apenas à presença de ingredientes básicos na sua despensa que poderiam ser incorporados.

4. **Camarão com caldo de moqueca (similaridade: 0.703):**  Esta receita não utiliza nenhum dos ingredientes que você possui em casa.

5. **Pavlova (similaridade: 0.696):**  Esta receita de sobremesa não contém nenhum dos ingredientes que você mencionou.


**Conclusão:**

A melhor opção, considerando os ingredientes que você tem em casa, é a **Feijoada**.  A receita de Nhoque à Bolonhesa *poderia* ser adaptada, mas exige mais ingredientes.  As outras sugestões são inadequadas.


**Receita de Feijoada (Passo a Passo):**

**Ingredientes:**

* 500g de feijão preto (de molho de um dia para o outro)
* 250g de carne de porco (costelinha, lombo, etc., em cubos)
* 250g de carne seca (dessalgada e cozida separadamente)
* 200g de linguiça calabresa (sem pele, fatiada)
* 1 cebola grande (picada)
* 4 dentes de alho (picados)
* 2 folhas de louro
* 2 colheres de sopa de azeite
* Sal e pimenta-do-reino a gosto
* Arroz branco cozido (para acompanhar)
* Farofa (opcional, para acompanhar)
* Laranja fatiada (opcional, para acompanhar)


**Modo de preparo:**

1. **Cozinhe o feijão:** Escorra o feijão e lave-o bem. Coloque em uma panela de pressão com água suficiente para cobrir e cozinhe por aproximadamente 30 minutos após pegar pressão.

2. **Refogue os temperos:** Em uma panela grande, aqueça o azeite. Adicione a cebola e o alho e refogue até dourar levemente.

3. **Adicione as carnes:** Junte a carne de porco e a linguiça à panela e frite até dourar.

4. **Misture tudo:** Adicione o feijão cozido (escorrido), a carne seca cozida, as folhas de louro, sal e pimenta à panela. Mexa bem.

5. **Cozinhe na panela de pressão:** Adicione mais água, se necessário, para cobrir a feijoada. Tampe a panela e cozinhe por mais 30 minutos sob pressão.

6. **Ajuste o tempero:** Após o tempo, abra a panela com cuidado. Experimente e ajuste o sal e a pimenta se necessário. Deixe ferver por mais alguns minutos sem a tampa, para reduzir o caldo.

7. **Sirva:** Sirva a feijoada quente com arroz branco, farofa (opcional) e fatias de laranja (opcional).


**Observações:**

* O tempo de cozimento do feijão e da feijoada pode variar dependendo da panela e do tipo de feijão utilizado.
* Você pode adicionar outros ingredientes à feijoada, como paio, orelha de porco, etc.
* A carne seca deve ser dessalgada adequadamente antes do uso.  Lembre-se de seguir as instruções da embalagem.


Bom apetite!


In [ ]:
sentence = "Quais receitas levam mais de 1 hora para preparar?"
similar_products = search_similar_products(sentence)
similar_products

[{'receita': 'Pavlova', 'similarity': 0.6756603717803955},
 {'receita': 'Feijoada', 'similarity': 0.67252516746521},
 {'receita': 'Boeuf Bourguignon', 'similarity': 0.6699164509773254},
 {'receita': 'Camarão com caldo de moqueca', 'similarity': 0.6658973097801208},
 {'receita': 'Nhoque à Bolonhesa', 'similarity': 0.6552871465682983}]

In [ ]:
prompt5 = """Quais receitas levam mais de 1 hora para preparar?
          Resultado:
        {'receita': 'Pavlova', 'similarity': 0.6756603717803955},
        {'receita': 'Feijoada', 'similarity': 0.67252516746521},
        {'receita': 'Boeuf Bourguignon', 'similarity': 0.6699164509773254},
        {'receita': 'Camarão com caldo de moqueca', 'similarity': 0.6658973097801208},
        {'receita': 'Nhoque à Bolonhesa', 'similarity': 0.6552871465682983}

Com base nas avaliações dos produtos apresentados, analise cada um deles em relação à sua busca.
"""

In [ ]:
response = modelGemini.generate_content(f"{prompt5}")
display(Markdown(response.text))

A busca era por receitas que levem mais de uma hora para preparar.  A lista apresentada, baseada em alguma métrica de similaridade (provavelmente relacionada a palavras-chave em descrições de receitas), sugere algumas opções, mas a similaridade não garante que todas levem *mais* de uma hora.  Algumas podem levar menos, dependendo da receita específica e da habilidade do cozinheiro.  Precisaríamos de mais informações para avaliar a precisão.

Vamos analisar cada uma:

* **Pavlova:**  Uma sobremesa elaborada com merengue.  A preparação do merengue e o tempo de cozimento no forno indicam que sim, provavelmente leva mais de uma hora, considerando o tempo de resfriamento. **Alta probabilidade de levar mais de 1 hora.**

* **Feijoada:** Um prato brasileiro de feijão preto com carnes diversas.  O tempo de cozimento do feijão e das carnes, especialmente se feitas do zero, com certeza ultrapassa uma hora. **Alta probabilidade de levar mais de 1 hora.**

* **Boeuf Bourguignon:** Um tradicional cozido de carne francês.  O longo período de cozimento em vinho tinto é fundamental para o sabor.  **Alta probabilidade de levar mais de 1 hora.**

* **Camarão com caldo de moqueca:** Moquecas, dependendo da receita, podem levar um tempo considerável para o preparo do caldo e o cozimento do camarão.  Considerando o preparo do caldo e a preparação dos ingredientes, **probabilidade moderada de levar mais de 1 hora**.  Receitas mais simplificadas poderiam levar menos.

* **Nhoque à Bolonhesa:** O nhoque em si leva um tempo razoável para ser feito (preparo da massa e cozimento), mas o molho bolonhesa, dependendo da receita, pode ser rápido.  A soma dos dois tempos pode ou não ultrapassar uma hora. **Baixa probabilidade de levar mais de 1 hora**, a menos que a bolonhesa seja feita do zero com um longo tempo de cozimento.


Em resumo: a lista parece razoável, mas a precisão depende da receita específica de cada item.  A similaridade usada como base não é uma garantia de tempo de preparo.  Para ter certeza, seria necessário consultar receitas detalhadas dessas opções.
